In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
!pip install catboost

In [3]:
train=pd.read_csv('../input/layerai-air-quality-prediction-challenge/train (13).csv')
test=pd.read_csv('../input/layerai-air-quality-prediction-challenge/test (14).csv')
sub=pd.read_csv('../input/layerai-air-quality-prediction-challenge/SampleSubmission (8).csv')

In [4]:
train.shape

(9923, 71)

In [5]:
test.drop('ID',axis=1, inplace=True)
train.drop('ID',axis=1, inplace=True)

In [6]:
new = train["date"].str.split("-", n = 2, expand = True)
train['year']=new[0].astype(int)
train['month']=new[1].astype(int)
train['day']=new[2].astype(int)
train=train.drop('date',axis=1)
new = test["date"].str.split("-", n = 2, expand = True)
test['year']=new[0].astype(int)
test['month']=new[1].astype(int)
test['day']=new[2].astype(int)
test=test.drop('date',axis=1)

In [7]:
coll=train.columns
train.shape

(9923, 72)

In [8]:
train.isna().sum().sort_values(ascending=False).head(65)

NitrogenDioxide_solar_zenith_angle                         6918
NitrogenDioxide_cloud_fraction                             6918
NitrogenDioxide_tropopause_pressure                        6918
NitrogenDioxide_NO2_slant_column_number_density            6918
NitrogenDioxide_stratospheric_NO2_column_number_density    6918
                                                           ... 
UvAerosolIndex_sensor_azimuth_angle                         335
UvAerosolIndex_solar_azimuth_angle                          335
UvAerosolIndex_sensor_zenith_angle                          335
temp_mean                                                    20
month                                                         0
Length: 65, dtype: int64

In [9]:
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [10]:
Y_train=train.pm2_5
X_train=train.drop('pm2_5',axis=1)
X_test=test
print(X_train.shape,X_test.shape,Y_train.shape)

(9923, 71) (4254, 71) (9923,)


In [11]:
X_train

,device,site_latitude,site_longitude,humidity,temp_mean,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,year,month,day
0,ANQ16PZJ,0.353465,32.560322,0.846844,22.038542,-0.000138,1.042629,-0.000143,0.291358,76.594910,...,3181.793457,7.799846,0.278111,76.618553,9.970293,-93.110069,19.214193,2020,3,18
1,AB6051M4,0.390741,32.582257,0.874493,21.545833,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,10,13
2,A0WN66FH,0.285751,32.578325,0.729340,23.568655,NaN,NaN,NaN,NaN,NaN,...,3440.390869,62.496590,0.416631,-97.300758,63.690605,-127.856804,40.857922,2019,12,21
3,aq_41,-1.244985,29.989236,0.919271,16.338542,-0.000111,0.750228,-0.000083,0.000000,75.789885,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2,20
4,aq_41,-1.244985,29.989236,0.858365,16.602083,-0.000127,0.792634,-0.000101,0.087338,74.787279,...,3842.057585,4.971117,0.277157,74.786025,32.583070,-57.273638,21.864529,2019,4,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9918,APZ80RTM,0.306666,32.608550,0.736872,22.965972,NaN,NaN,NaN,NaN,NaN,...,6156.016834,3.419584,0.235853,72.847686,50.858546,-31.908548,24.023105,2020,7,20
9919,aq_60,0.307780,32.651449,0.721721,26.555392,-0.000684,0.414260,-0.000283,0.249911,-97.272911,...,12266.058594,10.198909,0.284046,-97.272919,64.089607,-109.482361,32.003708,2020,2,23
9920,AJCK5L86,0.343898,32.587226,0.810413,22.915625,NaN,NaN,NaN,NaN,NaN,...,6754.737305,5.724823,0.299491,-98.203743,53.532288,-70.173080,32.261959,2020,4,17
9921,aq_76,0.357987,32.584100,0.876437,21.258333,NaN,NaN,NaN,NaN,NaN,...,5357.190430,49.711514,0.279414,71.634727,61.877548,-26.566027,25.409733,2020,6,8


In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(X_train['device'])
X_train['device']=le.transform(X_train['device'])
le.fit(X_test['device'])
X_test['device']=le.transform(X_test['device'])

In [13]:
minmax = MinMaxScaler()
trainn=pd.DataFrame(minmax.fit_transform(X_train), columns=X_train.columns)
test=pd.DataFrame(minmax.transform(X_test), columns=test.columns)

In [14]:
# mice = IterativeImputer(random_state=42, verbose=1)
# trainn=pd.DataFrame(mice.fit_transform(trainn),columns=trainn.columns)
# test=pd.DataFrame(mice.transform(test),columns=test.columns)

In [20]:

trainn = trainn.fillna(trainn.mean())
test = test.fillna(test.mean())

In [21]:
trainn

,device,site_latitude,site_longitude,humidity,temp_mean,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,year,month,day
0,0.151515,0.976771,0.930429,0.846844,0.741165,0.298563,0.541242,0.169229,0.971714,0.957219,...,0.158106,0.026330,0.269054,0.957273,0.149233,0.481256,0.322169,1.0,0.181818,0.566667
1,0.060606,0.999550,0.938367,0.874493,0.724595,0.354969,0.346135,0.219616,0.489922,0.462722,...,0.331150,0.067110,0.259875,0.464383,0.613738,0.525073,0.577421,0.0,0.818182,0.400000
2,0.000000,0.935393,0.936944,0.729340,0.792623,0.354969,0.346135,0.219616,0.489922,0.462722,...,0.178231,0.246216,0.409311,0.049216,0.961535,0.232322,0.966220,0.0,1.000000,0.666667
3,0.484848,0.000000,0.000000,0.919271,0.549472,0.307786,0.314098,0.188677,0.000000,0.952992,...,0.331150,0.067110,0.259875,0.464383,0.613738,0.525073,0.577421,1.0,0.090909,0.633333
4,0.484848,0.000000,0.000000,0.858365,0.558335,0.302200,0.347040,0.182940,0.291284,0.947728,...,0.209489,0.014958,0.268087,0.947705,0.491159,0.737997,0.401035,0.0,0.272727,0.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9918,0.212121,0.948174,0.947882,0.736872,0.772355,0.354969,0.346135,0.219616,0.489922,0.462722,...,0.389566,0.008720,0.226265,0.937585,0.767502,0.919719,0.465268,1.0,0.545455,0.633333
9919,0.818182,0.948854,0.963406,0.721721,0.893069,0.108640,0.053109,0.124331,0.833483,0.044318,...,0.865061,0.035974,0.275063,0.049361,0.967568,0.363961,0.702746,1.0,0.090909,0.733333
9920,0.090909,0.970925,0.940165,0.810413,0.770662,0.354969,0.346135,0.219616,0.489922,0.462722,...,0.436160,0.017988,0.290701,0.044501,0.807932,0.645583,0.710431,1.0,0.272727,0.533333
9921,0.969697,0.979535,0.939034,0.876437,0.714926,0.354969,0.346135,0.219616,0.489922,0.462722,...,0.327400,0.194819,0.270372,0.931252,0.934120,0.957994,0.506530,1.0,0.454545,0.233333


In [ ]:
l=['temp_mean','humidity','SulphurDioxide_SO2_column_number_density_amf','CarbonMonoxide_H2O_column_number_density','CarbonMonoxide_CO_column_number_density',
                        'NitrogenDioxide_absorbing_aerosol_index','UvAerosolIndex_absorbing_aerosol_index']



In [ ]:
# trainn["mean"]=trainn[l].mean(axis=1)
# trainn["count"]=trainn[l].count(axis=1)
# trainn["std"]=trainn[l].std(axis=1)
# trainn["min"]=trainn[l].min(axis=1)
# trainn["max"]=trainn[l].max(axis=1)
# trainn["kurt"]=trainn[l].kurt(axis=1)
# trainn["skew"]=trainn[l].skew(axis=1)
# trainn["q1"]=trainn[l].quantile(q=0.1,axis=1)
# trainn["q2"]=trainn[l].quantile(q=0.2,axis=1)
# trainn["q25"]=trainn[l].quantile(q=0.25,axis=1)
# trainn["q3"]=trainn[l].quantile(q=0.3,axis=1)
# trainn["q4"]=trainn[l].quantile(q=0.4,axis=1)
# trainn["q5"]=trainn[l].quantile(q=0.5,axis=1)
# trainn["q6"]=trainn[l].quantile(q=0.6,axis=1)
# trainn["q7"]=trainn[l].quantile(q=0.7,axis=1)
# trainn["q75"]=trainn[l].quantile(q=0.75,axis=1)
# trainn["q8"]=trainn[l].quantile(q=0.8,axis=1)
# trainn["q9"]=trainn[l].quantile(q=0.9,axis=1)


In [ ]:
# test["mean"]=test[l].mean(axis=1)
# test["count"]=test[l].count(axis=1)
# test["std"]=test[l].std(axis=1)
# test["min"]=test[l].min(axis=1)
# test["max"]=test[l].max(axis=1)
# test["kurt"]=test[l].kurt(axis=1)
# test["skew"]=test[l].skew(axis=1)
# test["q1"]=test[l].quantile(q=0.1,axis=1)
# test["q2"]=test[l].quantile(q=0.2,axis=1)
# test["q25"]=test[l].quantile(q=0.25,axis=1)
# test["q3"]=test[l].quantile(q=0.3,axis=1)
# test["q4"]=test[l].quantile(q=0.4,axis=1)
# test["q5"]=test[l].quantile(q=0.5,axis=1)
# test["q6"]=test[l].quantile(q=0.6,axis=1)
# test["q7"]=test[l].quantile(q=0.7,axis=1)
# test["q75"]=test[l].quantile(q=0.75,axis=1)
# test["q8"]=test[l].quantile(q=0.8,axis=1)
# test["q9"]=test[l].quantile(q=0.9,axis=1)

In [ ]:
# k=[]
# for i in coll :
#     if i!="pm2_5":
#         k.append(i)
#         # trainn.drop(k , axis=1 , inplace=True)
# # test.drop(k , axis=1 , inplace=True)

In [ ]:
# trainn = trainn.sort_values('device').reset_index(drop=True)

In [33]:
from sklearn.feature_selection import RFE
from catboost import CatBoostRegressor
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold,StratifiedKFold ,GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn import  ensemble
import xgboost as xgb
def get_model(Name='lgbm') :
    if Name=='lgbm' :
      return LGBMRegressor(**{'objective' :'regression','boosting_type' : 'gbdt','metric': 'rmse' ,
                              'learning_rate' : 0.05,'num_iterations': 1500,'max_depth' :8 ,'num_leaves' : 150,
                              'max_bins': 85,'min_data_in_leaf':30,'reg_lambda' :75})
col=X_train.columns
cat_model =CatBoostRegressor(loss_function = "RMSE", eval_metric = "RMSE",random_seed=123,
                                 learning_rate=0.1,  iterations=2000,verbose=100,
                                 bootstrap_type= "Poisson", 
                                 task_type="GPU")
test_ = test[col]
Model_Name = "lgbm"
kf = KFold(n_splits =10,shuffle=True,random_state=160)
# oofs  = np.zeros((len(X_train[col])))
test_predictions = np.zeros((len(X_test)))
gbm_predictions = []
cv_score_ = 0
oof_preds = np.zeros((X_train.shape[0],))

for fold_, (trn_idx, val_idx) in enumerate(kf.split(trainn, train['pm2_5'])):
    X_train, y_train = trainn.iloc[trn_idx][col], train.iloc[trn_idx]['pm2_5']
    X_valid, y_valid = trainn.iloc[val_idx][col], train.iloc[val_idx]['pm2_5']
#     X_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
#     X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    
    clf = get_model(Name=Model_Name)
    clf.fit(X_train, y_train, eval_set = [(X_valid, y_valid)],
            verbose =100, early_stopping_rounds = 200)
    Y_lgb = clf.predict(test_)
    test_predictions += Y_lgb / kf.n_splits


# #     model = get_model(Name="lgbm")
# #     rfe = RFE(estimator=model, n_features_to_select=71)

# #     pipeline = Pipeline(steps=[('s',rfe),('m',cat_model)])
# #     pipeline.fit(X_train, y_train)
#     gbm = xgb.XGBRegressor(eval_metric = 'rmse',n_estimators = 2000,learning_rate = 0.01,seed=162,random_state = 162,colsample_bytree=0.65)

#     gbm.fit(X_train,y_train,eval_set = [(X_valid, y_valid)],early_stopping_rounds  = 200,verbose=100)
  
#     cv_score_ += mean_squared_error(y_valid, gbm.predict(X_valid), squared=False) / kf.n_splits
#     oof_preds[val_idx] = gbm.predict(X_valid)
  
#     preds = gbm.predict(test_)
#     gbm_predictions.append(preds)
    



    
# #     vp = clf.predict(X_valid)
# #     oofs[val_idx] = vp
# #     val_score = mean_squared_error((vp), (y_valid),squared=True)
# #     print(4*'-- -- -- --')
# #     print(f'Fold {fold_+1} Val score: {val_score}')
# #     print(4*'-- -- -- --')
    
# #     tp = clf.predict(test_)


  
# print()
# print(3*'###',10*"^",3*'###')
# # print(mean_squared_error(train["pm2_5"], oofs,squared=True))
# print("Model training")
# # clf.fit(X_train, y_train )

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[100]	valid_0's rmse: 15.5471
[200]	valid_0's rmse: 14.106
[300]	valid_0's rmse: 13.4637
[400]	valid_0's rmse: 13.2382
[500]	valid_0's rmse: 13.1092
[600]	valid_0's rmse: 13.016
[700]	valid_0's rmse: 12.9629
[800]	valid_0's rmse: 12.9163
[900]	valid_0's rmse: 12.888
[1000]	valid_0's rmse: 12.8605
[1100]	valid_0's rmse: 12.8338
[1200]	valid_0's rmse: 12.8102
[1300]	valid_0's rmse: 12.7764
[1400]	valid_0's rmse: 12.7637
[1500]	valid_0's rmse: 12.7512
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[100]	valid_0's rmse: 12.9785
[200]	valid_0's rmse: 11.3174
[300]	valid_0's rmse: 10.7728
[400]	valid_0's rmse: 10.5849
[500]	valid_0's rmse: 10.495
[600]	valid_0's rmse: 10.4708
[700]	valid_0's rmse: 10.4562
[800]	valid_0's rmse: 10.4335
[900]	valid_0's rmse: 10.448
[1000]	valid_0's rmse: 10.4414
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[100

In [ ]:
# cat_modell =CatBoostRegressor(loss_function = "RMSE", eval_metric = "RMSE",random_seed=123)

# model = get_model(Name="lgbm")
# rfe = RFE(estimator=model, n_features_to_select=71)
# gbm = xgb.XGBRegressor(eval_metric = 'rmse',n_estimators = 2000,learning_rate = 0.01,seed=162,random_state = 162,colsample_bytree=0.65)


# pipeline = Pipeline(steps=[('s',rfe),("xgb", gbm)])
# pipeline.fit(trainn, train["pm2_5"])
# Y_lgb = pipeline.predict(test_)

In [34]:
sub['pm2_5']=test_predictions 
sub

,ID,pm2_5
0,ID_UOH62J0XHX,35.077565
1,ID_4OPWSB0UHJ,107.773940
2,ID_3SLMNNG1Z3,64.781286
3,ID_MFUHTXYPM4,67.833684
4,ID_TY1DAND8ZP,82.062366
...,...,...
4249,ID_NYKGOYFBKZ,52.836341
4250,ID_IWO4EVLHSQ,69.897011
4251,ID_G1M1IMY9WU,55.174005
4252,ID_DXV65Q1QXI,65.731437


In [30]:
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
model = get_model(Name="lgbm")
rfe = RFE(estimator=model, n_features_to_select=71)
from catboost import CatBoostRegressor
cat_model = CatBoostRegressor(depth=8,iterations=4000)
pipeline = Pipeline(steps=[('s',rfe),('m',cat_model)])
pipeline.fit(trainn, train["pm2_5"])
Y_lgb = pipeline.predict(test)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
Learning rate set to 0.019062
0:	learn: 27.2776496	total: 130ms	remaining: 8m 40s
1:	learn: 27.0701879	total: 180ms	remaining: 6m
2:	learn: 26.8835299	total: 233ms	remaining: 5m 10s
3:	learn: 26.6652020	total: 288ms	remaining: 4m 47s
4:	learn: 26.4831563	total: 340ms	remaining: 4m 32s
5:	learn: 26.3013925	total: 395ms	remaining: 4m 22s
6:	learn: 26.1084129	total: 448ms	remaining: 4m 15s
7:	learn: 25.9251774	total: 502ms	remaining: 4m 10s
8:	learn: 25.7499517	total: 556ms	remaining: 4m 6s
9:	learn: 25.5850894	total: 609ms	remaining: 4m 2s
10:	learn: 25.4288843	total: 662ms	remaining: 3m 59s
11:	learn: 25.2858688	total: 713ms	remaining: 3m 56s
12:	learn: 25.0980079	total: 765ms	remaining: 3m 54s
13:	learn: 24.9527009	total: 818ms	remaining: 3m 52s
14:	learn: 24.8088161	total: 875ms	remaining: 3m 52s
15:	learn: 24.6682757	total: 928ms	remaining: 3m 51s
16:	learn: 24.53

In [31]:
sub['pm2_5']=Y_lgb 
sub

,ID,pm2_5
0,ID_UOH62J0XHX,41.235296
1,ID_4OPWSB0UHJ,130.832163
2,ID_3SLMNNG1Z3,62.531007
3,ID_MFUHTXYPM4,74.138019
4,ID_TY1DAND8ZP,75.052926
...,...,...
4249,ID_NYKGOYFBKZ,45.308760
4250,ID_IWO4EVLHSQ,81.232640
4251,ID_G1M1IMY9WU,63.629714
4252,ID_DXV65Q1QXI,65.600455


In [35]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")
create_submission(sub, "sol")

/kaggle/working/sol.csv

In [ ]:
sub

****

In [ ]:
train=train.drop('pm2_5',axis=1)

In [ ]:
train=pd.concat([train,test])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
col_scal=train.iloc[:,3:-3].columns
train[col_scal]=minmax.fit_transform(train[col_scal])


In [ ]:
train['device']=train['device'].apply(lambda x:str(x))
train['ID']=train['ID'].apply(lambda x:str(x))

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(train['device'])
train['device']=le.transform(train['device'])

le.fit(train['ID'])
train['ID']=le.transform(train['ID'])

In [ ]:
nan_train=train.columns[train.isnull().any(axis=0)]
nan_train

In [ ]:
temp_=['temp_mean']
Sulph=nan_train[1:10]
Carbo=nan_train[10:18]
Nitro=nan_train[18:30]
Forma=nan_train[30:38]
UvAer=nan_train[38:44]
Ozone=nan_train[44:53]
Cloud=nan_train[53:64]
list_rp=[temp_,UvAer,Ozone,Cloud,Carbo,Sulph,Forma,Nitro]


In [ ]:
train.drop("date", axis=1, inplace=True)

In [ ]:
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression

lightgbm = LGBMRegressor(objective='regression', 
                       num_leaves=8,
                       learning_rate=0.05, 
                       n_estimators=1000,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.5,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1)
D_ind={}
D_nn={}
D_rp={}
for i in list_rp:
  name=i[0][:5]
  train_nan=train.loc[train[i[0]].isnull()]
  D_ind[name]=train_nan.index
  train_pred=train.drop(train_nan.index)
  D_nn[(name,'train')]=train_pred
  D_nn[(name,'test')]=train_nan
  new_list = [x for x in nan_train if (x not in i)]
  mean_rp=D_nn[(i[0][:5],'train')][new_list].mean()
  D_rp[i[0][:5]]=mean_rp
  for p in new_list:
    D_nn[(i[0][:5],'train')][p]=D_nn[(i[0][:5],'train')][p].fillna(method='ffill')
    D_nn[(i[0][:5],'test')][p]=D_nn[(i[0][:5],'test')][p].fillna(method='ffill')
  for j in i:
    y_train=D_nn[(i[0][:5],'train')][j]
    x_train=D_nn[(i[0][:5],'train')].drop(columns=i)
    x_test=D_nn[(i[0][:5],'test')].drop(columns=i)
    print(x_train)
    model =lightgbm
    model.fit(x_train,y_train)
    y_cat=model.predict(x_test)
    x_test[j]=y_cat
    x_train[j]=y_train
    print(1,i[0][:5],y_cat.mean(),y_train.mean())

    for val in list(x_test.ID):
      train.loc[train.ID==val,j]=list(x_test.loc[x_test.ID==val,j])[0]

In [ ]:
lightgbm = LGBMRegressor(objective='regression', 
                       num_leaves=6,
                       learning_rate=0.05, 
                       n_estimators=1000,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.5,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1)
D_comp={}
for i in list_rp:
  name=i[0][:5]
  D_comp[name]=train.iloc[list(D_ind[name])]
  for j in D_ind[name]:
    for l in i:
      train[l].iloc[j]=np.nan
  train_nan=train.loc[train[i[0]].isnull()]
  D_ind[name]=train_nan.index
  train_pred=train.drop(train_nan.index)
  D_nn[(name,'train')]=train_pred
  D_nn[(name,'test')]=train_nan

  for j in i:
    y_train=D_nn[(i[0][:5],'train')][j]
    x_train=D_nn[(i[0][:5],'train')].drop(columns=i)
    x_test=D_nn[(i[0][:5],'test')].drop(columns=i)

    model =lightgbm
    model.fit(x_train,y_train)
    y_cat=model.predict(x_test)
    x_test[j]=y_cat
    x_train[j]=y_train

    print(2,i[0][:5],y_cat.mean(),y_train.mean())
    for val in list(x_test.ID):
      train.loc[train.ID==val,j]=list(x_test.loc[x_test.ID==val,j])[0]
  train.info()


In [ ]:
Data=pd.DataFrame([[0]*len(test.columns)],columns=test.columns)
Data=Data.drop(0)
Data

In [ ]:
Data1=pd.DataFrame([[0]*len(test.columns)],columns=test.columns)
Data1=Data1.drop(0)
Data1

In [ ]:
len(train.ID.unique())

In [ ]:
train.ID=le.inverse_transform(list(train.ID))

In [ ]:
for id in list(train.ID):
  if id not in list(test["ID"]):
    Data1=pd.concat([Data1,train.loc[train.ID==id]])
Data1

In [ ]:
for id in test["ID"]:
  Data=pd.concat([Data,train.loc[train.ID==id]])
Data

In [ ]:
tr=pd.read_csv('../input/layerai-air-quality-prediction-challenge/train (13).csv')
pm_train=tr['pm2_5']
X_train=Data1
X_test=Data
Y_train=pm_train

In [ ]:
X_train.shape

In [ ]:
X_train['pred']=Y_train
X_train.corr().pred.sum()

In [ ]:
X_train.corr()['pred']

In [ ]:
s=0
for i in list(X_train.corr()['pred'][1:]):
    s+=abs(i)
s

In [ ]:
X_train=X_train.drop('pred',axis=1)

In [ ]:
todrop=[]
for i in X_train.corr().pred.keys():
  if abs(X_train.corr().pred[i])<0.05:
    todrop+=[i]

In [ ]:
X_train=X_train.drop('pred',axis=1)

In [ ]:
X_train=X_train.drop(columns=todrop)
X_test=X_test.drop(columns=todrop)

In [ ]:
from sklearn import preprocessing
le1 = preprocessing.LabelEncoder()

le1.fit(X_train['ID'])
X_train['ID']=le1.transform(X_train['ID'])
le1.fit(X_test['ID'])
X_test['ID']=le1.transform(X_test['ID'])

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
model = get_model(Name="lgbm")
rfe = RFE(estimator=model, n_features_to_select=71)
from catboost import CatBoostRegressor
cat_model = CatBoostRegressor(depth=8,iterations=4000)
pipeline = Pipeline(steps=[('s',rfe),('m',cat_model)])
pipeline.fit(X_train, Y_train)
Y_lgb = pipeline.predict(X_test)

In [ ]:
lightgbm = LGBMRegressor(objective='regression', 
                       num_leaves=20,
                       learning_rate=0.09, 
                       n_estimators=10000,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.5,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1,
                       random_state=42)

In [ ]:
lightgbm.fit(X_train,Y_train)

In [ ]:
Y_lgb=model.predict(X_test)
Y_lgb

In [ ]:
sub['pm2_5']=Y_lgb 
sub

In [ ]:

from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")
create_submission(sub, "sol")

****

****

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train,Y_train)
Y_pred_lr=model.predict(X_test)
Y_pred_lr

In [ ]:
cat_model = CatBoostRegressor()
cat_model.fit(x_train, y_train)

In [ ]:
#build model
from catboost import CatBoostRegressor


In [ ]:
D0={}
for i in nan_train:
  if i not in zero_train: 
    mean_0=int(train.loc[train[i]==0]['pm2_5'].mean())
    a=train.loc[(train.pm2_5<mean_0*1.1) & (train.pm2_5>mean_0*0.9)][['pm2_5',i]][i].sum()
    b=train.loc[(train.pm2_5<mean_0*1.1) & (train.pm2_5>mean_0*0.9)][['pm2_5',i]][i].value_counts().iloc[1:].sum()
    rp_0=a/b
    train[i]=train[i].replace(0,rp_0)
    D0[i]=rp_0
D0

In [ ]:
for i in zero_train:
  pm_mean=train.loc[train[i].isnull()].pm2_5.mean()
  mean_rp=train.loc[train.pm2_5>pm_mean-1].loc[train.pm2_5<pm_mean+1][i].mean()
  D0[i]=mean_rp
D0

In [ ]:
for i in nan_train:
  if i not in zero_train:
    train[i]=train[i].replace(0,D0[i])
  else :
    train[i]=train[i].replace(np.nan,D0[i])



In [ ]:
for i in nan_test:
  if i not in zero_test:
    test[i]=test[i].replace(0,D0[i])
  else :
    test[i]=test[i].replace(np.nan,D0[i])



In [ ]:
for i in nan_train:
  if train.loc[train[i]==0].shape[0]>0:
    print(i)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(X_train['ID'])
X_train['ID']=le.transform(X_train['ID'])

le.fit(X_test['ID'])
X_test['ID']=le.transform(X_test['ID'])

In [ ]:
X_test

In [ ]:
Y=train['pm2_5']
X=train.drop('pm2_5',axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=2)

In [ ]:
print(X_train.shape, X_test.shape, Y_train.shape)

In [ ]:
X_train=X
Y_train=Y
X_test=test

In [ ]:
train.corr().iloc[:20]

In [ ]:
import pandas as pd 
X_train=pd.read_csv('/content/X_train.csv')
X_test=pd.read_csv('/content/X_test.csv')
X_test

In [ ]:
Y_train=pm_train

In [ ]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


In [ ]:
from sklearn.linear_model import Ridge

clf_ridge = Ridge(alpha=0.05)
clf_ridge.fit(X_train,Y_train)
Y_pred_rid=clf_ridge.predict(X_test)
Y_pred_rid

In [ ]:
X_test.groupby('device').count()

In [ ]:

import xgboost as xgb

xgb = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
xgb.fit(X_train,Y_train)
Y_xgb=xgb.predict(X_test)


In [ ]:
pip install catboost

In [ ]:
#build model
from catboost import CatBoostRegressor

cat_model = CatBoostRegressor(depth=8,iterations=4000)
# Fit model 
cat_model.fit(X_train, Y_train)
Y_cat=cat_model.predict(X_test)


In [ ]:
Y_cat

In [ ]:
acc=metrics.mean_absolute_error(Y_cat,Y_pred_xgb)
acc

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor


In [ ]:
# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(13, input_shape=(13,), kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model
# evaluate model
estimator = KerasRegressor(model=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold, scoring='neg_mean_squared_error')
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

# SUBMIISION


In [ ]:
Res=[]
for i in range(sub.shape[0]):
  Res+=[[Id_test[i],Y_lgb[i]]]
Res